## Clasificación - Parte 1. Funciones Discriminantes

Primero, corremos la celda de preparación.

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "06_Regularizacion"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "plots", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
# import warnings
# warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Discriminante lineal de Fischer

Veamos cómo el criterio de Fischer nos permite encontrar la dirección óptima para realizar la proyección de las instancias (datos).

Creemos un set aleatorio en dos dimensiones, compuesto de dos clases, cuya distribución es multinormal

$$
p(x | C_k) = \mathcal{N}(\mu_k, \Sigma_k)\;\;.
$$

Por supuesto, si supieramos a priori que esta es la distribución, podríamos usar las técnicas que vamos a ver la próxima, para modelos discriminativos.

Elijamos parámetros a ojo, y muestremos elementos de cada clase.

In [ ]:
#from scipy.stats import multivariate_normal
from numpy.random import multivariate_normal

size1 = 250
mu1 = [2, 1]
cov1 = [[1, 0.95],[0.95, 1]]

size2 = 200
mu2 = [1, 3]
cov2 = [[1, 0.2],[0.2, 1]]

# Sample classes
xc1 = multivariate_normal(mean=mu1, cov=cov1, size=size1).T

xc2 = multivariate_normal(mean=mu2, cov=cov2, size=size2).T

In [ ]:
# Veamos cómo se ven
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

ax.plot(*xc1, 'ob', mfc='None', label='C1')
ax.plot(*xc2, 'or', mfc='None', label='C2')

ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.legend(loc='lower right', fontsize=16)
ax.set_aspect('equal')

Claramente, si proyectamos, digamos sobre el eje de $x_2$, las clases no se separan muy bien.

In [ ]:
# Veamos cómo se ven
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

nbins = 20
ax.hist(xc1[1], nbins, histtype='step', color='b')
ax.hist(xc2[1], nbins, histtype='step', color='r')

ax.set_xlabel('$y$')

Veamos si podemos mejorar esto usando el criterio de Fischer. Para eso, tenemos que calcular la matriz de covarianza intra-clase

$$
\mathbf{S}_I = \underbrace{\sum_{n\,\in\,\mathcal{C}_1} \left(\mathbf{x}_n - \mathbf{m}_1\right)\left(\mathbf{x}_n - \mathbf{m}_1\right)^\mathrm{T}}_{\text{Matriz de covarianza intra-clase 1}}
+ \underbrace{\sum_{n\,\in\,\mathcal{C}_2} \left(\mathbf{x}_n - \mathbf{m}_2\right)\left(\mathbf{x}_n - \mathbf{m}_2\right)^\mathrm{T}}_{\text{Matriz de covarianza intra-clase 2}}\;\;,
$$
donde
$$
\mathbf{m}_i = \frac{1}{N_i}\sum_{n\,\in\,\mathcal{C}_i} \mathbf{x}_n
$$

In [ ]:
m = np.zeros([2,2])
Si = np.zeros([2, 2, 2])
for i, x in enumerate([xc1, xc2]):
    # Calcula media empírica
    m[i] = np.mean(x, axis=1)
    xm = (x.T - m[i]).T
    Si[i] = np.matmul(xm,xm.T)

    print('Covariance Matrix for class {}\n'.format(i+1), Si[i])

In [ ]:
# Sumemos ahora para obetener la matriz total
S = np.sum(Si, axis=0)
print('Covariance Matrix Total\n', S)

Ahora resolvemos el problema lineal

$$
S_I \mathbf{w} = \mathbf{m}_2 - \mathbf{m}_1
$$

para obtener $\mathbf{w}$.

In [ ]:
w = np.linalg.solve(S, (m[1] - m[0])[:, np.newaxis])

# Normalizamos, por amor al arte
w /= np.linalg.norm(w)
print(w)

In [ ]:
# Agregemos el vector al plot
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

ax.plot(*xc1, 'ob', mfc='None', label='C1')
ax.plot(*xc2, 'or', mfc='None', label='C2')

# Ploteo vector de pesos
ax.quiver([0], [0], w[0], w[1], color='green', scale=10)

# ploteo plano perpendicular
xp = np.array([-1.5, 4])
yp = -w[0] * xp / w[1]

plt.plot(xp, yp, 'o:k')

ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.legend(loc='lower right', fontsize=16)
ax.set_aspect('equal')

Vamos a ver si me sale proyectar en la dirección perpendicular a $\mathbf{w}$. Para eso, obtengo:

$$
\mathbf{y} = \mathbf{w}^\mathrm{T} \mathbf{x}
$$

In [ ]:
# Encuentro proyección de cada clase
yc1 = np.dot(w.T, xc1)
yc2 = np.dot(w.T, xc2)

In [ ]:
# Hagamos nuevamente el histograma
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

nbins = 20
ax.hist(yc1[0], nbins, histtype='step', color='b')
ax.hist(yc2[0], nbins, histtype='step', color='r')

ax.set_xlabel('$y$ óptimo')

## Perceptron

Vimos cómo funciona el perceptron. Ahora vamos a usarlo para clasificar datos.

### Datos de MNIST

Vamos a usar un set de datos clásico de Machine Learning, los números de MNIST. Se trata de 70 000 imágenes pequeñas de dígitos escritos a mano. El "target" de cada uno de estos dígitos es el número que representan.

Este set de datos es tan común, que en <tt>sklearn</tt> hay una función que permite bajarlos directamente. Dependiendo de la versión que <tt>sklearn</tt> que estén usando, la función relevante del paquete <tt>datasets</tt> cambia. Además, cambia la forma en la que devuelven los datos (antes estaban ordenados por valor del *target*, ahora viene así nomás. Para que el resultado sea idéntico con ambas versiones, usamos este código, que nos presta amablemente Géron.

In [ ]:
def sort_by_target(mnist):
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:, 1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000]
    mnist.target[60000:] = mnist.target[reorder_test + 60000]

In [ ]:
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
    mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings
    sort_by_target(mnist) # fetch_openml() returns an unsorted dataset
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')

El resultado es un tipo que todavía no habíamos visto. No vamos a entrar en detalles, pero digamos que tiene los datos en el atributo data y los valores de los labels en el atributo target.

In [ ]:
print(mnist.data.shape)
print(mnist.target.shape)

Los datos tienen 784 *features*, que corresponden a cada uno de los píxeles de las imágenes de 28 x 28. El valor oscila entre 0 (blanco) y 255 (negro). Veamos:

In [ ]:
print(mnist.data[500])

Separemos ahora datos de labels. Seguimos usando nuestra notación, en la que los labels se llaman *t*.

In [ ]:
X, t = mnist["data"], mnist["target"]

In [ ]:
# Agarremos un dígito cualquiera
un_digito = X[36000]

# Lo ponemos en forma de imagen y lo vemos.
un_digito_image = un_digito.reshape(28, 28)
plt.imshow(some_digit_image, cmap = mpl.cm.binary,
           interpolation="None")
plt.axis("off")

#save_fig("some_digit_plot")
plt.show()

Es un cinco (creo). Confirmemos.

In [ ]:
t[36000]

Lo que hicimos recién para plotear el número está piola. Vamos a convertirlo en una función para tener a mano.

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

Ahora veamos varios números. Usamos otro código que nos vuelve a prestar nuestro amigo Aurélien.

In [ ]:
# EXTRA
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9))
example_images = np.r_[X[:12000:600], X[13000:30600:600], X[30600:60000:590]]
plot_digits(example_images, images_per_row=10)
#save_fig("more_digits_plot")
plt.show()

Antes de seguir mirando, tenemos que separar el conjunto en entrenamiento y testeo. Por suerte, los datos MNIST ya vienen separados, de forma de tener buena representación de cada clase. Las primeras 60000 instancias son de entrenamiento y las últimas 10000 de testeo.

In [ ]:
X_train, X_test, t_train, t_test = X[:60000], X[60000:], t[:60000], t[60000:]

Vamos a barajar el conjunto de entrenamiento. Esto es para asegurarse de que todos los dígitos estarán bien representandos en distintos *folds* de validación cruzada. (Ver clase anterior y [notebook](06_Regularización.ipynb)).

In [ ]:
import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, t_train = X_train[shuffle_index], t_train[shuffle_index]

### Clasificación binaria

Vamos a empezar haciendo las cosas fáciles y dividir el problema. Vamos a intentar detectar "cincos". Para eso, generamos un nuevo label, que sea 1 cuando el número es cinco, y cero cuando no lo sea. Obviamente, hacemos lo mismo para el test.

In [ ]:
t_train_5 = (t_train == 5)
t_test_5 = (t_test == 5)

<tt>sklearn</tt> tiene una clase <tt>Perceptron</tt>, que es la que vamos a usar.

In [ ]:
from sklearn.linear_model import Perceptron

perce = Perceptron()

# Hagamos un fit usando como features los valores de los píxeles directamente (linear regression)
# En ese caso, la matriz de diseño es simplemente, Xtrain
phi = X_train.copy()

# Pero podríamos probar otras cosas, como esto que está comentado más abajo.
#phi = np.log(1 + X_train.copy()**2)

perce = perce.fit(phi, t_train_5)

In [ ]:
print('Perceptron says', perce.predict([un_digito]))

Bueno, parece que ese cinco raro lo identifica correctamente. Pero claro, estaba dentro del conjunto de entrenamiento. Esto no quiere decir absolutamente nada.

***
**Pregunta**: ¿O sí?
***

In [ ]:
from sklearn.model_selection import cross_val_score
#cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
scores = cross_val_score(perce, X_train, t_train_5, cv=5, scoring="accuracy")
print(scores)
print(scores.mean())

Esto parece increíble. Más del 96% de "accuracy", con un modelo lineal muy simple. ¿Es posible?

***
**Pregunta**: Pensemos un poco más en detenimiento, considerando la naturaleza del dataset. ¿Es realmente un valor tan alto? ¿Qué pasaría si hicieramos un clasificador que dijera que el número nunca es cinco?
***

En efecto, en este tipo de datasets, donde los datos están muy desbalanceados, no es muy útil la "accuracy". Es mejor usar la matriz de confusión.

Para crearla, neceistamos calcular predicciones en cada uno de nuestras instancias de entrenamiento, si queremos evitar usar el conjunto de test. En ese caso, podemos hacer CV y calcular las predicciones en cada uno de los folds. Eso lo hace una función de <tt>sklearn</tt> (gracias por tanto!), aunque la implementación no sería complicada.

In [ ]:
from sklearn.model_selection import cross_val_predict

t_train_pred = cross_val_predict(perce, phi, t_train_5, cv=5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(t_train_5, t_train_pred)
print(cm)

Con estos valores en mano, podemos calcular el exhaustividad (*recall*) y la precisión.

Recordemos:

$$
\mathrm{precision} = \frac{TP}{TP + FP}
$$

$$
\mathrm{recall} = \frac{TP}{TP + FN}
$$

donde TP son los *true positives* (es decir, la cantidad de casos relevantes recuperados correctamente), FP son los *falsos positivos* (es decir, la cantidad de casos no relevante recuperados incorrectamente), y FN son los *falsos negativos* (es decir, la cantidad de casos relevantes **no** recuperados).

Calculemos estas cosas

In [ ]:
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]

print('Precision: ', tp/(tp + fp))
print('Recall: ',  tp/(tp + fn))

Estos resultados hay que interpretarlos de la siguiente manera: la precisión nos dice en qué fracción de los casos en los que perceptrón dice que tiene un "5", realmente lo tiene. Además, el *recall* nos dice que fracción de todos los "5" encontró.

Esto todavía no lo vimos, pero uno puede, en principio, ajustar estos valores mirando el nivel del umbral que usa para clasificar una instancia como verdadera o no. Como vimos, por defecto esto es $y(\mathbf{x}) = 0$ para el perceptron, pero podemos cambiarlo. Para eso, necesitamos el perceptron nos diga el valor de $y(\mathbf{x_k})$, para cada imagen $x_k$

In [ ]:
# Podríamos hacer así
y = perce.decision_function(X_train)

# Pero mejor sería hacerlo con CV, usando 5 folds
y = cross_val_predict(perce, X_train, t_train_5, cv=3, method="decision_function")

In [ ]:
# Veamos cómo se ven
A = plt.hist(y, 100)
plt.xlabel('y(x)')

Ahora hagamos un código que vaya variando el umbral y nos calcule la precisión y el recall.

In [ ]:
umb = np.linspace(y.min()-1, y.max()+1, 5000)

recall = np.zeros_like(umb)
precision = np.zeros_like(umb)
fpr = np.zeros_like(umb)

for i, u in enumerate(umb):
    
    # Calcula los índices con detecciones para este umbral
    det = y > u
    
    # Compara esto con los verdaderos casos en esos índices

    tp = np.sum(t_train_5[det] == True)
    # Falsos positivos
    fp = np.sum(t_train_5[det] == False)

    # Verdaderos y falsos negativos
    tn = np.sum(t_train_5[~det] == False)
    fn = np.sum(t_train_5[~det] == True)
    
    recall[i] = tp/(tp + fn)
    precision[i] = tp/(tp + fp)
    fpr[i] = fp/(fp + tn)

In [ ]:
fig = plt.figure(figsize=(12, 5))
plt.plot(umb, precision, label='Precisión', lw=2)
plt.plot(umb, recall, label='Recall', lw=2)
plt.xlabel("Umbral", fontsize=16)
plt.legend(loc=0, fontsize=16)
plt.xlim(-umb.max(), umb.max())
plt.axvline(0.0, ls=':', color='0.5')

Se puede plotear directo uno vs. el otro:

In [ ]:
fig = plt.figure(figsize=(7, 7))
plt.plot(precision, recall)
plt.axvline(0.1, ls=':', color='0.5')
plt.xlabel('Precisión')
plt.ylabel('Recall')
plt.title('Curva PR')

Otra forma de ver esto es con la curva ROC (Receiver Operating Characteristic, o Característica Operativa del Receptor), que es muy similar, salvo que plotea la tasa de verdaderos positivos (es decir, el recall, en función de la tasa de falsos positivos). Cuanto más alejado de la recta unidad esté el sistema mejor, pero por supuesto esto depende del problema a resolver.

In [ ]:
fig = plt.figure(figsize=(7, 7))
plt.plot(fpr, recall)
plt.plot([0, 1], [0, 1], color='0.5', ls=':')
plt.xlabel('Tasa de falsos positivos (FPR)')
plt.ylabel('Tasa de verdaderos positivos (TPR) / Recall')
plt.title('Curva ROC')

Otra característica interesante es el area bajo la curva. Podemos hacer una estimación veloz, sumando:

In [ ]:
np.sum(recall[1:] * np.diff(fpr))

Hay código de <tt>sklearn</tt> para todo esto.

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
precisions, recalls, thresholds = precision_recall_curve(t_train_5, y)
fpr, tpr, thresholds = roc_curve(t_train_5, y)
print(roc_auc_score(t_train_5, y))